# Классификатор, для определения положения изображения в мозгу

используем пока всё то же самое, чт ов регрессоре, кроме последнего слоя сетки

пока сделаю в лоб 24 класса 

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
import os
import numpy as np
import nibabel as nib
from nilearn import image
import matplotlib.pyplot as plt

In [2]:
path = r'F:\!наука\stroke\Lesions\Lesions'
stroke_types = os.listdir(path)
X = []
Y = []
for folder in stroke_types:
    for patient in os.listdir(f'{path}\{folder}'):
        img = nib.load(f'{path}\{folder}\{patient}\dwi.nii.gz')
        if len(img.shape) == 4:
            img = image.index_img(img, 0)
            img_data = img.get_fdata()
            length = img.shape[2] if img.shape[2] <=25 else 25
            for i in range(length):
                Y.append(i)
                X.append(img_data[:, :, i])


In [3]:
X = np.array(X)
Y = np.array(Y).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [4]:
#Конвертируем в тензоры

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

In [5]:
# создаём датасеты и лоадеры

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

#### Модель

Сейчас пробуем достаточно простую:

- Сверточный слой out channels 16
- Пулинг с ядром 2
- Relu
- Сверточный слой out channels 32
- Пулинг с ядром 2
- Relu
- Сверточный слой out channels 64
- Пулинг с ядром 2
- Relu
- дропаут 20%
- Полносвязный на 128 признаков
- Полносвязный  на 1 выход

В качестве активации используем ReLU


In [6]:
import torch.nn as nn

class CNNClassifier(nn.Module):
    def __init__(self, num_classes=24):
        super(CNNClassifier, self).__init__()
        
        #изображение чб, так что на вход 1 канал. Попробуем применить 16 фильтров 
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # получает на вход 16 каналов и выдаёт 32
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1)
        #дропнем ещё 20% фич
        self.drop = nn.Dropout2d(p=0.2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1)
        # Наша картинка размером 256х256 и к ней 3 раза применяется пулинг с ядром размера 2
        # 256/2/2/2 = 32. Значит, наш тензор признаков будет 32х32 а мы нагенерировали их 128
        self.fc1 = nn.Linear(in_features=32 * 32 * 128, out_features=num_classes)
        
    
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = nn.functional.dropout(self.drop(x), training=self.training)

        # Flatten
        x = x.view(-1, 32 * 32 * 128)
        x = self.fc1(x)


        return torch.log_softmax(x, dim=1)


#### Тренировочка

В качестве лосса будем использовать MSE, в качестве оптимизатора - Adam

In [7]:
device = "cpu"
if (torch.cuda.is_available()):
    # if GPU available, use cuda (on a cpu, training will take a considerable length of time!)
    device = "cuda"

model = CNNClassifier(num_classes=24).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(10):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        #обнуляем градиенты
        optimizer.zero_grad()

        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backpropagate
        loss.backward()
        optimizer.step()

        #выведем статистику
        if (batch_idx+1)%10 == 0:
            print(f'Epoch: {epoch+1}/10, Step: {batch_idx+1}/{len(train_loader)}, Loss: {loss.item() :.2f}')


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
#ну и на тесте
with torch.no_grad(): #тут нам градиент не нужен, не тратим на него время
    total_loss = 0
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()
    mean_loss = total_loss / len(test_loader)
    print(f'Test Loss: {mean_loss:.4f}')

Test Loss: 503.1458


In [ ]:
outputs.T.shape

torch.Size([1, 31])

In [ ]:
targets.T

tensor([[197., 133., 194.,  87., 144., 146., 141.,  74., 165., 104., 179., 129.,
         141., 169., 217.,  63., 219.,  77.,  95., 100., 143., 100.,  92., 133.,
         139.,  72.,  74.,  87.,  15.,  66., 119.,  92.]], device='cuda:0')